In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from MFDFA import MFDFA
from MFDFA import fgn

# A fractional Ornstein−Uhlenbeck (fOU) process

A simple Ornstein--Uhlenbeck process takes the form
$$\mathrm{d}y(t) = -\theta y(t)\mathrm{d}t + \sigma \mathrm{d}B_H(t), $$
with $\theta$ the drift coefficient, $\sigma$ diffusion term, and $B(t)$ a fractional Brownian motion with index $H$.

There is a Note at the end of this notebook on fractional Gaussian noise and fractional Brownian motion, if you wish to understand it a bit better.

## Integrating the fOU with an Euler−Maruyama scheme

In [ ]:
# The total integration time
t_final = 500

# The desired timestep of integration
delta_t = 0.001

# time array of the process
time = np.linspace(0, t_final, t_final * int(1 / delta_t))

# Choose some values for the drift and diffusion
theta = 0.3
sigma = 0.1

# Generate your favourite fractional Gaussian noise with H index
H = 0.7
dB = (t_final ** H) * fgn(N = time.size, H = H)

# Initialise the array y
y = np.zeros([time.size])

# Give some small random initial conditions
y[0]=np.random.normal(size = 1) / 10

# Integrate the process
for i in range(1, time.size):
    y[i] = y[i-1] - theta * y[i-1] * delta_t + sigma * dB[i] 

## Visualising the process

In [ ]:
#This is the stochastic trajectory over time
plt.plot(time, y, label = r'Trajectory of fOU process')

plt.xlabel(r'time $t$')
plt.ylabel(r'$y(t)$')
plt.legend()

# Employing MFDFA

Here we will implement the MFDFA to try and recover the Hurst index $H$ for the generated fOU process.

To employ MFDFA we will need a sequence of segment lengths, *lag*, and a selection of powers $q$. Let's start with $q=2$, which is simply DFA. Moreover we need to select the order of the polynomial fittings, which we'll take as straight lines, i.e., 1-st order polynomials.

In [ ]:
# Select a band of lags, which usually ranges from 
# very small segments of data, to very long ones, as
lag = np.logspace(0.7, 4, 30).astype(int)
# Notice these must be ints, since these will segment
# the data into chucks of lag size

# Select the power q
q = 2

# The order of the polynomial fitting
order = 1

# Obtain the (MF)DFA as
lag, dfa = MFDFA(y, lag = lag, q = q, order = order)

## Understanding MFDFA

To actually understand MFDFA we need to study the results in log-log plots and find some slopes

In [ ]:
# To uncover the Hurst index, lets get some log-log plots
plt.loglog(lag, dfa, 'o', label='fOU: MFDFA q=2')

# And now we need to fit the line to find the slope. We will
# fit the first points, since the results are more accurate 
# there. Don't forget that if you are seeing in log-log
# scales, you need to fit the logs of the results
np.polyfit(np.log(lag[:15]), np.log(dfa[:15]),1)[0]

# Now what you should obtain is: slope = H + 1

Why $H + 1$? Well, the Euler−Maruyama scheme literally integrates the process: Integration implies + 1 (it increases the regularity by 1).

Curious about other processes? You can just input $dB_H$ into the MFDFA and see what you get (it will be $H$ the slope, since this is a *noise*). You can instead put np.cumsum($dB_H$) and you will get slope = $H+1$ (since it is a *motion*)

## Side note on fractional Gaussian noise

Fractional Brownian motion is an extension of Brownian motion that allows to have correlations in the noise. You can visualise its effects and characteristics by simply considering $B_H(t)$ for different $H$ values. The regular Brownian motion has an $H$ index of $1/2$.

In [ ]:
# Lets take three examples, with H=0.3, H=0.5, H=0.7
# The total integration time, as before
t_final = 500

# The desired timestep of integration
delta_t = 0.001

# time array of the process
time = np.linspace(0, t_final, t_final * int(1 / delta_t))

# Generate three fractional Gaussian noises dB 
H_anti = 0.3       # Anti-presistent noise
H_regu = 0.5       # Regular noise
H_posi = 0.7       # Positively correlated noise

# Generate the noises (with the appropriate normalisation)
dB_anti = (t_final ** H_anti) * fgn(N = time.size, H = H_anti)
dB_regu = (t_final ** H_regu) * fgn(N = time.size, H = H_regu)
dB_posi = (t_final ** H_posi) * fgn(N = time.size, H = H_posi)

In [ ]:
# Let's plot the noises, and the associated motions

fig, ax = plt.subplots(2,3, figsize=(12,4));

ax[0,0].plot(time, dB_anti)
ax[0,1].plot(time, dB_regu)
ax[0,2].plot(time, dB_posi)

# their motions are given by the integral of the noise,
# i.e., the cumsum of the process

ax[1,0].plot(time, np.cumsum(dB_anti))
ax[1,1].plot(time, np.cumsum(dB_regu))
ax[1,2].plot(time, np.cumsum(dB_posi))

As you can see, the first path $H=0.3$ is very *rough*, because $H<1/2$ means anti-correlation: If on the previous step the noise when up, it is more likely to go down!

For $H=0.5$ is just the regular Brownian motion.

For positively correlated noise $H=0.7$, you have the opposite, if you just went up, you are likely to continue going up, so you get *smooth* paths.